In [ ]:
from __future__ import print_function
import os
import time
import logging
import argparse
import numpy as np
from visdom import Visdom
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from utils import *
from metric.loss import FitNet, AttentionTransfer, RKdAngle, RkdDistance

# Teacher models:
# VGG11/VGG13/VGG16/VGG19, GoogLeNet, AlxNet, ResNet18, ResNet34, 
# ResNet50, ResNet101, ResNet152, ResNeXt29_2x64d, ResNeXt29_4x64d, 
# ResNeXt29_8x64d, ResNeXt29_32x64d, PreActResNet18, PreActResNet34, 
# PreActhttps://www.bing.com/?mkt=zh-CNResNet50, PreActResNet101, PreActResNet152, 
# DenseNet121, DenseNet161, DenseNet169, DenseNet201, 
import models

# Student models:
# myNet, LeNet, FitNet

start_time = time.time()
# os.makedirs('./checkpoint', exist_ok=True)

# Training settings
parser = argparse.ArgumentParser(description='PyTorch ada. FitNet')

parser.add_argument('--dataset',
                    choices=['CIFAR10',
                             'CIFAR100'
                            ],
                    default='CIFAR10')
parser.add_argument('--teachers',
                    choices=['ResNet32',
                             'ResNet50',
                             'ResNet56',
                             'ResNet110',
                             'DenseNet121'
                            ],
                    default=['ResNet32', 'ResNet56', 'ResNet110'],
                    nargs='+')
parser.add_argument('--student',
                    choices=['ResNet8',
                             'ResNet15',
                             'ResNet16',
                             'ResNet20',
                             'myNet'
                            ],
                    default='ResNet20')

parser.add_argument('--kd_ratio', default=0.7, type=float)
parser.add_argument('--n_class', type=int, default=10, metavar='N', help='num of classes')
parser.add_argument('--T', type=float, default=20.0, metavar='Temputure', help='Temputure for distillation')
parser.add_argument('--batch_size', type=int, default=128, metavar='N', help='input batch size for training')
parser.add_argument('--test_batch_size', type=int, default=128, metavar='N', help='input test batch size for training')
parser.add_argument('--epochs', type=int, default=20, metavar='N', help='number of epochs to train (default: 20)')
parser.add_argument('--lr', type=float, default=0.1, metavar='LR', help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M', help='SGD momentum (default: 0.5)')
parser.add_argument('--device', default='cuda:0', type=str, help='device: cuda or cpu')
parser.add_argument('--print_freq', type=int, default=10, metavar='N', help='how many batches to wait before logging training status')

config = ['--epochs', '200', '--T', '5.0', '--device', 'cuda:0']
args = parser.parse_args(config)

device = args.device if torch.cuda.is_available() else 'cpu'
load_dir = './checkpoint/' + args.dataset + '/'

# teachers model
teacher_models = []
for te in args.teachers:
    te_model = getattr(models, te)(num_classes=args.n_class)
#     print(te_model)
    te_model.load_state_dict(torch.load(load_dir + te_model.model_name + '.pth'))
    te_model.to(device)
    teacher_models.append(te_model)

st_model = getattr(models, args.student)(num_classes=args.n_class)  # args.student()
st_model.to(device)

# logging
logfile = load_dir + 'ada_fitnet_' + st_model.model_name + '.log'
if os.path.exists(logfile):
    os.remove(logfile)
def log_out(info):
    f = open(logfile, mode='a')
    f.write(info)
    f.write('\n')
    f.close()
    print(info)
    
# visualizer
vis = Visdom(env='distill')
loss_win = vis.line(
    X=np.array([0]),
    Y=np.array([0]),
    opts=dict(
        title='FitNet ada. loss',
        xtickmin=0,
#         xtickmax=1,
#         xtickstep=5,
        ytickmin=0,
#         ytickmax=1,
        ytickstep=0.5,
#         markers=True,
#         markersymbol='dot',
#         markersize=5,
    ),
    name="loss"
)

acc_win = vis.line(
    X=np.column_stack((0, 0)),
    Y=np.column_stack((0, 0)),
    opts=dict(
        title='FitNet ada. ACC',
        xtickmin=0,
#         xtickstep=5,
        ytickmin=0,
        ytickmax=100,
#         markers=True,
#         markersymbol='dot',
#         markersize=5,
        legend=['train_acc', 'test_acc']
    ),
    name="acc"
)


# adapter model
class Adapter():
    def __init__(self, in_models, pool_size):
        # representations of teachers
        pool_ch = pool_size[1]  # 64
        pool_w = pool_size[2]   # 8
        LR_list = []
        torch.manual_seed(1)
        self.theta = torch.randn(len(in_models), pool_ch).to(device)  # [3, 64]
        self.theta.requires_grad_(True)
   
        self.max_feat = nn.MaxPool2d(kernel_size=(pool_w, pool_w), stride=pool_w).to(device)
        self.W = torch.randn(pool_ch, 1).to(device)
        self.W.requires_grad_(True)
        self.val = False

    def loss(self, y, labels, weighted_logits, T=10.0, alpha=0.7):
        ls = nn.KLDivLoss()(F.log_softmax(y/T), weighted_logits) * (T*T * 2.0 * alpha) + F.cross_entropy(y, labels) * (1. - alpha)
        if not self.val:
            ls += 0.1 * (torch.sum(self.W * self.W) + torch.sum(torch.sum(self.theta * self.theta, dim=1), dim=0))
        return ls
        
    def gradient(self, lr=0.01):
        self.W.data = self.W.data - lr * self.W.grad.data
        # Manually zero the gradients after updating weights
        self.W.grad.data.zero_()
        
    def eval(self):
        self.val = True
        self.theta.detach()
        self.W.detach()
    
    # input size: [64, 8, 8], [128, 3, 10]
    def forward(self, conv_map, te_logits_list):
        beta = self.max_feat(conv_map)
        beta = torch.squeeze(beta)  # [128, 64]
        
        latent_factor = []
        for t in self.theta:
            latent_factor.append(beta * t)
#         latent_factor = torch.stack(latent_factor, dim=0)  # [3, 128, 64]
        alpha = []
        for lf in latent_factor:  # lf.size:[128, 64]
            alpha.append(lf.mm(self.W))
        alpha = torch.stack(alpha, dim=0)  # [3, 128, 1]
        alpha = torch.squeeze(alpha).transpose(0, 1) # [128, 3]
        weight = F.softmax(alpha)  # [128, 3]

        return weight

# adapter instance
_,_,_,pool_m,_ = st_model(torch.randn(1,3, 128, 128).to(device))  # get pool_size of student
# reate adapter instance
adapter = Adapter(teacher_models, pool_m.size())


# data
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, 4),
    transforms.ToTensor(),
    normalize,
])
test_transform = transforms.Compose([transforms.ToTensor(), normalize])
train_set = getattr(datasets, args.dataset)(root='../data', train=True, download=True, transform=train_transform)
test_set = getattr(datasets, args.dataset)(root='../data', train=False, download=False, transform=test_transform)
train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=args.test_batch_size, shuffle=False)
# optim
optimizer_W = optim.SGD([adapter.W], lr=args.lr, momentum=0.9)
optimizer_theta = optim.SGD([adapter.theta], lr=args.lr, momentum=0.9)
optimizer_sgd = optim.SGD(st_model.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer_sgd, gamma=0.1, milestones=[100, 150])
lr_scheduler2 = optim.lr_scheduler.MultiStepLR(optimizer_W, milestones=[40, 50])
lr_scheduler3 = optim.lr_scheduler.MultiStepLR(optimizer_theta, milestones=[40, 50])

# loss
dist_criterion = RkdDistance().to(device)
angle_criterion = RKdAngle().to(device)
fitnet_criterion = [FitNet(32, 64), FitNet(64, 64),FitNet(64, 64)]
[f.to(device) for f in fitnet_criterion]


def train_adapter(n_epochs=70, model=st_model):
    print('Training adapter:')
    start_time = time.time()
    model.train()
    adapter.eval()
    for ep in range(n_epochs):
        lr_scheduler2.step()
        lr_scheduler3.step()
        for i, (input, target) in enumerate(train_loader):

            input, target = input.to(device), target.to(device)
            # compute outputs
            b1, b2, b3, pool, output = model(input) # out_feat: 16, 32, 64, 64, - 
            st_maps = [b1, b2, b3, pool]
#             print('b1:{}, b2:{}, b3{}, pool:{}'.format(b1.size(), b2.size(), b3.size(), pool.size()))
# b1:torch.Size([128, 16, 32, 32]), b2:torch.Size([128, 32, 16, 16]), b3torch.Size([128, 64, 8, 8]), pool:torch.Size([128, 64, 1, 1])

            te_scores_list = []
            hint_maps = []
            fit_loss = 0
            for j,te in enumerate(teacher_models):
                te.eval()
                with torch.no_grad():
                    t_b1, t_b2, t_b3, t_pool, t_output = te(input)
#                 print('t_b1:{}, t_b2:{}, t_b3:{}, t_pool:{}'.format(t_b1.size(), t_b2.size(), t_b3.size(), t_pool.size()))
# t_b1:torch.Size([128, 16, 32, 32]), t_b2:torch.Size([128, 32, 16, 16]), t_b3:torch.Size([128, 64, 8, 8]), t_pool:torch.Size([128, 64, 1, 1])
                hint_maps.append(t_pool)
                t_output = F.softmax(t_output/args.T)
                te_scores_list.append(t_output)
            te_scores_Tensor = torch.stack(te_scores_list, dim=1)  # size: [128, 3, 10]
            
            weight = adapter.forward(pool, te_scores_Tensor)
            weight_t = torch.unsqueeze(weight, dim=2)
            weighted_logits = weight_t * te_scores_Tensor  # [128, 3, 10]
            weighted_logits = torch.sum(weighted_logits, dim=1)
            weight_f = F.softmax(torch.mean(weight, dim=0))
            
            optimizer_sgd.zero_grad()
            optimizer_W.zero_grad()
            optimizer_theta.zero_grad()
            
            angle_loss = angle_criterion(output, weighted_logits)
            dist_loss = dist_criterion(output, weighted_logits)
            # compute gradient and do SGD step
            ada_loss = adapter.loss(output, target, weighted_logits, T=args.T, alpha=args.kd_ratio)
            
            for j in range(len(teacher_models)-1):
                fit_loss += fitnet_criterion[j](st_maps[j+1], hint_maps[j]) #weight_f[j] * 
#             fit_loss = fitnet_criterion[0](b2, hint_maps[0][3]) + fitnet_criterion[1](b3, hint_maps[1][3]) + fitnet_criterion(pool, hint_maps[2][3])
            loss = ada_loss + fit_loss #+ dist_loss + angle_loss
            
            loss.backward(retain_graph=True)
            optimizer_sgd.step()
            optimizer_W.step()
            optimizer_theta.step()
            
#          vis.line(np.array([loss.item()]), np.array([ep]), loss_win, update="append")
        log_out('epoch[{}/{}]adapter Loss: {:.4f}'.format(ep, n_epochs, loss.item()))
    end_time = time.time()
    log_out("--- adapter training cost {:.3f} mins ---".format((end_time - start_time)/60))


# train with multi-teacher
def train(epoch, model):
    print('Training:')
    # switch to train mode
    model.train()
    adapter.eval()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    
    end = time.time()
    for i, (input, target) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.to(device), target.to(device)
        
        # compute outputs
        b1, b2, b3, pool, output = model(input)
        st_maps = [b1, b2, b3, pool]
        
        te_scores_list = []
        hint_maps = []
        fit_loss = 0
        for j,te in enumerate(teacher_models):
            te.eval()
            with torch.no_grad():
                t_b1, t_b2, t_b3, t_pool, t_output = te(input)
        
            hint_maps.append(t_pool)
            t_output = F.softmax(t_output/args.T)
            te_scores_list.append(t_output)
        te_scores_Tensor = torch.stack(te_scores_list, dim=1)  # size: [128, 3, 10]
        
        weight = adapter.forward(pool, te_scores_Tensor)
        weight_t = torch.unsqueeze(weight, dim=2)
        weighted_logits = weight_t * te_scores_Tensor  # [128, 3, 10]
        weighted_logits = torch.sum(weighted_logits, dim=1)
        weight_f = F.softmax(torch.mean(weight, dim=0))
        
        optimizer_sgd.zero_grad()
        
        angle_loss = angle_criterion(output, weighted_logits)
        dist_loss = dist_criterion(output, weighted_logits)
        
        # compute gradient and do SGD step
        ada_loss = adapter.loss(output, target, weighted_logits, T=args.T, alpha=args.kd_ratio)
        for j in range(len(teacher_models)-1):
            fit_loss += fitnet_criterion[j](st_maps[j+1], hint_maps[j])
        
        loss = ada_loss + fit_loss #+ dist_loss + angle_loss

        loss.backward(retain_graph=True)
        optimizer_sgd.step()

        output = output.float()
        loss = loss.float()
        # measure accuracy and record loss
        train_acc = accuracy(output.data, target.data)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(train_acc, input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            log_out('[{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1))
    return losses.avg, train_acc.cpu().numpy()


def test(model):
    print('Testing:')
    # switch to evaluate mode
    model.eval()
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input, target = input.to(device), target.to(device)

            # compute output
            _,_,_,_,output = model(input)
            loss = F.cross_entropy(output, target)

            output = output.float()
            loss = loss.float()

            # measure accuracy and record loss
            test_acc = accuracy(output.data, target.data)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(test_acc, input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                log_out('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(test_loader), batch_time=batch_time, loss=losses,
                          top1=top1))

    log_out(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))

    return losses.avg, test_acc.cpu().numpy(), top1.avg.cpu().numpy()

# """
print('StudentNet:\n')
print(st_model)
st_model.apply(weights_init_normal)
train_adapter(n_epochs=80)
# st_model.apply(weights_init_normal)
best_acc = 0
for epoch in range(1, args.epochs + 1):
    log_out("\n===> epoch: {}/{}".format(epoch, args.epochs))
    log_out('current lr {:.5e}'.format(optimizer_sgd.param_groups[0]['lr']))
    lr_scheduler.step(epoch)
    train_loss, train_acc = train(epoch, st_model)
    # visaulize loss
    vis.line(np.array([train_loss]), np.array([epoch]), loss_win, update="append")
    _, test_acc, top1 = test(st_model)
    vis.line(np.column_stack((train_acc, top1)), np.column_stack((epoch, epoch)), acc_win, update="append")
    if top1 > best_acc:
        best_acc = top1
            
# release GPU memory
torch.cuda.empty_cache()
log_out("BEST ACC: {:.3f}".format(best_acc))
log_out("--- {:.3f} mins ---".format((time.time() - start_time)/60))
# """

/home/data/yaliu/jupyterbooks/multi-KD/models/teacher/resnet20.py:36: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight)


Files already downloaded and verified
StudentNet:

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2

/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:248: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:183: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:256: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/torch/nn/_reduction.py:15: UserWarning: reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.
  warnings.warn("reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.")
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launc

epoch[0/80]adapter Loss: 1.6207
epoch[1/80]adapter Loss: 1.3406
epoch[2/80]adapter Loss: 1.1132
epoch[3/80]adapter Loss: 0.9417
epoch[4/80]adapter Loss: 0.8863
epoch[5/80]adapter Loss: 1.0321
epoch[6/80]adapter Loss: 0.7275
epoch[7/80]adapter Loss: 0.6101
epoch[8/80]adapter Loss: 0.7095
epoch[9/80]adapter Loss: 0.8646
epoch[10/80]adapter Loss: 0.7112
epoch[11/80]adapter Loss: 0.6941
epoch[12/80]adapter Loss: 0.8521
epoch[13/80]adapter Loss: 0.6496
epoch[14/80]adapter Loss: 0.5462
epoch[15/80]adapter Loss: 0.6668
epoch[16/80]adapter Loss: 0.7586
epoch[17/80]adapter Loss: 0.4472
epoch[18/80]adapter Loss: 0.5592
epoch[19/80]adapter Loss: 0.5962
epoch[20/80]adapter Loss: 0.4775
epoch[21/80]adapter Loss: 0.6044
epoch[22/80]adapter Loss: 0.5636
epoch[23/80]adapter Loss: 0.4450
epoch[24/80]adapter Loss: 0.7001
epoch[25/80]adapter Loss: 0.5509
epoch[26/80]adapter Loss: 0.4664
epoch[27/80]adapter Loss: 0.5320
epoch[28/80]adapter Loss: 0.6596
epoch[29/80]adapter Loss: 0.5473
epoch[30/80]adapter 

/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:315: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:323: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

[60/391]	Time 0.183 (0.152)	Data 0.039 (0.038)	Loss 0.5627 (0.4843)	Prec@1 85.156 (86.629)
[70/391]	Time 0.186 (0.152)	Data 0.063 (0.039)	Loss 0.4377 (0.4853)	Prec@1 92.188 (86.565)
[80/391]	Time 0.180 (0.152)	Data 0.055 (0.039)	Loss 0.4618 (0.4861)	Prec@1 89.844 (86.526)
[90/391]	Time 0.164 (0.154)	Data 0.039 (0.039)	Loss 0.4893 (0.4845)	Prec@1 88.281 (86.684)
[100/391]	Time 0.148 (0.153)	Data 0.029 (0.039)	Loss 0.4757 (0.4807)	Prec@1 92.188 (86.920)
[110/391]	Time 0.165 (0.153)	Data 0.036 (0.038)	Loss 0.3960 (0.4807)	Prec@1 92.188 (86.909)
[120/391]	Time 0.171 (0.153)	Data 0.049 (0.039)	Loss 0.7037 (0.4835)	Prec@1 84.375 (86.900)
[130/391]	Time 0.174 (0.153)	Data 0.044 (0.039)	Loss 0.5689 (0.4846)	Prec@1 85.156 (86.838)
[140/391]	Time 0.136 (0.153)	Data 0.034 (0.038)	Loss 0.4238 (0.4850)	Prec@1 88.281 (86.708)
[150/391]	Time 0.148 (0.153)	Data 0.039 (0.038)	Loss 0.5453 (0.4857)	Prec@1 85.938 (86.698)
[160/391]	Time 0.136 (0.153)	Data 0.032 (0.038)	Loss 0.4113 (0.4842)	Prec@1 89.062 (

[10/391]	Time 0.133 (0.124)	Data 0.046 (0.037)	Loss 0.5561 (0.4444)	Prec@1 85.938 (88.565)
[20/391]	Time 0.178 (0.132)	Data 0.050 (0.035)	Loss 0.4427 (0.4656)	Prec@1 88.281 (87.574)
[30/391]	Time 0.158 (0.140)	Data 0.036 (0.037)	Loss 0.3749 (0.4658)	Prec@1 89.844 (87.450)
[40/391]	Time 0.136 (0.142)	Data 0.023 (0.036)	Loss 0.5203 (0.4679)	Prec@1 88.281 (87.595)
[50/391]	Time 0.169 (0.146)	Data 0.037 (0.036)	Loss 0.6479 (0.4623)	Prec@1 79.688 (87.883)
[60/391]	Time 0.158 (0.147)	Data 0.048 (0.037)	Loss 0.5078 (0.4663)	Prec@1 85.938 (87.666)
[70/391]	Time 0.139 (0.146)	Data 0.027 (0.036)	Loss 0.4675 (0.4704)	Prec@1 86.719 (87.335)
[80/391]	Time 0.158 (0.147)	Data 0.039 (0.036)	Loss 0.3824 (0.4692)	Prec@1 91.406 (87.365)
[90/391]	Time 0.132 (0.146)	Data 0.033 (0.036)	Loss 0.5312 (0.4660)	Prec@1 81.250 (87.509)
[100/391]	Time 0.142 (0.146)	Data 0.036 (0.037)	Loss 0.4674 (0.4669)	Prec@1 88.281 (87.430)
[110/391]	Time 0.138 (0.146)	Data 0.034 (0.037)	Loss 0.4443 (0.4712)	Prec@1 85.938 (87.26

Test: [40/79]	Time 0.033 (0.039)	Loss 1.5837 (1.3121)	Prec@1 66.406 (72.199)
Test: [50/79]	Time 0.041 (0.038)	Loss 1.3296 (1.2980)	Prec@1 71.875 (72.457)
Test: [60/79]	Time 0.035 (0.039)	Loss 0.9588 (1.3066)	Prec@1 79.688 (72.515)
Test: [70/79]	Time 0.044 (0.038)	Loss 1.2082 (1.2991)	Prec@1 76.562 (72.766)
 * Prec@1 72.840

===> epoch: 39/200
current lr 1.00000e-01
Training:
[0/391]	Time 0.177 (0.177)	Data 0.040 (0.040)	Loss 0.4595 (0.4595)	Prec@1 86.719 (86.719)
[10/391]	Time 0.124 (0.146)	Data 0.031 (0.040)	Loss 0.5559 (0.4819)	Prec@1 82.031 (87.500)
[20/391]	Time 0.112 (0.148)	Data 0.037 (0.042)	Loss 0.4056 (0.4674)	Prec@1 89.844 (87.723)
[30/391]	Time 0.158 (0.153)	Data 0.041 (0.042)	Loss 0.4373 (0.4728)	Prec@1 88.281 (87.550)
[40/391]	Time 0.183 (0.150)	Data 0.044 (0.039)	Loss 0.3546 (0.4636)	Prec@1 91.406 (87.900)
[50/391]	Time 0.152 (0.152)	Data 0.038 (0.040)	Loss 0.4209 (0.4631)	Prec@1 87.500 (88.006)
[60/391]	Time 0.137 (0.153)	Data 0.028 (0.040)	Loss 0.5139 (0.4635)	Prec@1 88

[380/391]	Time 0.142 (0.150)	Data 0.051 (0.039)	Loss 0.4654 (0.4828)	Prec@1 86.719 (87.061)
[390/391]	Time 0.151 (0.150)	Data 0.052 (0.039)	Loss 0.4035 (0.4832)	Prec@1 86.250 (87.042)
Testing:
Test: [0/79]	Time 0.036 (0.036)	Loss 0.8753 (0.8753)	Prec@1 82.031 (82.031)
Test: [10/79]	Time 0.036 (0.035)	Loss 1.5283 (1.0207)	Prec@1 71.875 (76.918)
Test: [20/79]	Time 0.039 (0.036)	Loss 1.0634 (1.1188)	Prec@1 80.469 (76.339)
Test: [30/79]	Time 0.041 (0.037)	Loss 0.7629 (1.0949)	Prec@1 78.125 (76.462)
Test: [40/79]	Time 0.040 (0.039)	Loss 1.3979 (1.0664)	Prec@1 72.656 (77.153)
Test: [50/79]	Time 0.034 (0.039)	Loss 0.8051 (1.0538)	Prec@1 79.688 (77.191)
Test: [60/79]	Time 0.034 (0.040)	Loss 1.4290 (1.0476)	Prec@1 75.000 (77.241)
Test: [70/79]	Time 0.032 (0.039)	Loss 0.9584 (1.0440)	Prec@1 77.344 (77.212)
 * Prec@1 77.290

===> epoch: 41/200
current lr 1.00000e-01
Training:
[0/391]	Time 0.140 (0.140)	Data 0.037 (0.037)	Loss 0.4678 (0.4678)	Prec@1 90.625 (90.625)
[10/391]	Time 0.139 (0.145)	Data

[330/391]	Time 0.143 (0.149)	Data 0.030 (0.038)	Loss 0.4928 (0.4903)	Prec@1 88.281 (86.728)
